Import All Required Libraries

In [242]:
import requests
import pandas as pd
import sqlite3
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
from datetime import datetime
from IPython.display import display, HTML

Loading Environmental Variable

In [243]:
# Load environment variables (for API key)
load_dotenv()

# API Configuration
BASE_URL = "https://api.sportradar.com/tennis/trial/v3/en/"
API_KEY = os.getenv("SPORTRADAR_API_KEY")  # Store your API key in .env file

Create Sqlite DataBase

In [244]:
# Create a SQLite database
def create_database():
    """Create SQLite database with the required schema"""
    conn = sqlite3.connect('tennis_analytics.db')
    cursor = conn.cursor()
    
    # Create Categories table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Categories (
        category_id VARCHAR(50) PRIMARY KEY,
        category_name VARCHAR(100) NOT NULL
    )
    ''')
    
    # Create Competitions table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Competitions (
        competition_id VARCHAR(50) PRIMARY KEY,
        competition_name VARCHAR(100) NOT NULL,
        parent_id VARCHAR(50),
        type VARCHAR(20) NOT NULL,
        gender VARCHAR(10) NOT NULL,
        category_id VARCHAR(50),
        FOREIGN KEY (category_id) REFERENCES Categories(category_id)
    )
    ''')
    
    # Create Complexes table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Complexes (
        complex_id VARCHAR(50) PRIMARY KEY,
        complex_name VARCHAR(100) NOT NULL
    )
    ''')
    
    # Create Venues table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Venues (
        venue_id VARCHAR(50) PRIMARY KEY,
        venue_name VARCHAR(100) NOT NULL,
        city_name VARCHAR(100) NOT NULL,
        country_name VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        timezone VARCHAR(100) NOT NULL,
        complex_id VARCHAR(50),
        FOREIGN KEY (complex_id) REFERENCES Complexes(complex_id)
    )
    ''')
    
    # Create Competitors table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Competitors (
        competitor_id VARCHAR(50) PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        country VARCHAR(100) NOT NULL,
        country_code CHAR(3) NOT NULL,
        abbreviation VARCHAR(10) NOT NULL
    )
    ''')
    
    # Create Competitor_Rankings table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Competitor_Rankings (
        rank_id INTEGER PRIMARY KEY AUTOINCREMENT,
        rank INT NOT NULL,
        movement INT NOT NULL,
        points INT NOT NULL,
        competitions_played INT NOT NULL,
        competitor_id VARCHAR(50),
        FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id)
    )
    ''')
    
    conn.commit()
    conn.close()
    
    print("Database created successfully!")

In [245]:
def fetch_api_data(endpoint):
    """Fetch data from SportRadar API"""
    url = f"{BASE_URL}/{endpoint}.json?api_key={API_KEY}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None

In [246]:
# Function to collect competitions data
def collect_competitions_data():
    """Collect and store competitions data"""
    data = fetch_api_data("competitions") 
    
    
    if not data:
        print("Failed to fetch competitions data")
        return
    
    conn = sqlite3.connect('tennis_analytics.db')
    cursor = conn.cursor()
    
    # Process categories
    for competition in data.get('competitions', []):
        category = competition.get('category', {})
        category_id = category.get('id')
        category_name = category.get('name')
        
        # Insert category
        cursor.execute('''
        INSERT OR IGNORE INTO Categories (category_id, category_name)
        VALUES (?, ?)
        ''', (category_id, category_name))
        
        # Process competitions within this category
    for competition in data.get('competitions', []):       
        competition_id = competition.get('id')
        competition_name = competition.get('name')
        parent_id = competition.get('parent_id')
        competition_type = competition.get('type')
        gender = competition.get('gender')
        category = competition.get('category', {})
        category_id = category.get('id')            

        # Insert competition
        cursor.execute('''
        INSERT OR IGNORE INTO Competitions 
        (competition_id, competition_name, parent_id, type, gender, category_id)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (competition_id, competition_name, parent_id, competition_type, gender, category_id))            
                    
    
    conn.commit()
    conn.close()
    
    print("Competitions data collected and stored successfully!")


In [247]:
# Function to collect complexes data
def collect_complexes_data():
    """Collect and store complexes data"""
    data = fetch_api_data("complexes")
    
    if not data:
        print("Failed to fetch complexes data")
        return
    
    conn = sqlite3.connect('tennis_analytics.db')
    cursor = conn.cursor()
    
    # Process complexes
    for complex_data in data.get('complexes', []):
        complex_id = complex_data.get('id')
        complex_name = complex_data.get('name')
        
        # Insert complex
        cursor.execute('''
        INSERT OR IGNORE INTO Complexes (complex_id, complex_name)
        VALUES (?, ?)
        ''', (complex_id, complex_name))
        
        # Process venues within this complex
        for venue in complex_data.get('venues', []):
            venue_id = venue.get('id')
            venue_name = venue.get('name')
            city_name = venue.get('city_name')
            country_name = venue.get('country_name')
            country_code = venue.get('country_code')
            timezone = venue.get('timezone')
            
            # Insert venue
            cursor.execute('''
            INSERT OR IGNORE INTO Venues 
            (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id))
    
    conn.commit()
    conn.close()
    
    print("Complexes data collected and stored successfully!")


In [248]:
def collect_doubles_rankings_data():
    """Collect and store doubles competitor rankings data"""
    data = fetch_api_data("double_competitors_rankings")
    
    if not data:
        print("Failed to fetch doubles rankings data")
        return
    
    conn = sqlite3.connect('tennis_analytics.db')
    cursor = conn.cursor()
    
    # Process competitors and their rankings
    for ranking in data.get('rankings', []):
        for comp_ranking in ranking.get('competitor_rankings', []):

            competitor = comp_ranking.get('competitor',{})
            competitor_id=competitor.get('id')
            name=competitor.get('name')
            country=competitor.get('country')
            country_code=competitor.get('country_code')
            abbreviation = competitor.get('abbreviation')
            
            # Insert competitor
            cursor.execute('''
            INSERT OR IGNORE INTO Competitors 
            (competitor_id, name, country, country_code, abbreviation)
            VALUES (?, ?, ?, ?, ?)
            ''', (competitor_id, name, country, country_code, abbreviation))
            
            # Insert ranking
            rank = comp_ranking.get('rank')
            movement = comp_ranking.get('movement')
            points = comp_ranking.get('points')
            competitions_played = comp_ranking.get('competitions_played')
            
            cursor.execute('''
            INSERT INTO Competitor_Rankings 
            (rank, movement, points, competitions_played, competitor_id)
            VALUES (?, ?, ?, ?, ?)
            ''', (rank, movement, points, competitions_played, competitor_id))
    
    conn.commit()
    conn.close()
    
    print("Doubles rankings data collected and stored successfully!")

In [249]:
# Function to execute SQL queries and display results
def execute_query(query, description=""):
    """Execute SQL query and display results"""
    conn = sqlite3.connect('tennis_analytics.db')
    result = pd.read_sql_query(query, conn)
    conn.close()
    
    print(f"\n{description}:")
    display(result)
    return result


In [250]:
# Competitions SQL Queries
def run_competitions_queries():
    """Run and display results for competitions SQL queries"""
    # Query 1: List all competitions along with their category name
    query1 = """
    SELECT c.competition_id, c.competition_name, cat.category_name
    FROM Competitions c
    JOIN Categories cat ON c.category_id = cat.category_id
    """
    result1 = execute_query(query1, "List all competitions along with their category name")
    
    # Query 2: Count the number of competitions in each category
    query2 = """
    SELECT cat.category_name, COUNT(c.competition_id) as competition_count
    FROM Competitions c
    JOIN Categories cat ON c.category_id = cat.category_id
    GROUP BY cat.category_name
    ORDER BY competition_count DESC
    """
    result2 = execute_query(query2, "Count the number of competitions in each category")
    
    # Query 3: Find all competitions of type 'doubles'
    query3 = """
    SELECT competition_id, competition_name, gender
    FROM Competitions
    WHERE type = 'doubles'
    """
    result3 = execute_query(query3, "Find all competitions of type 'doubles'")
    
    # Query 4: Get competitions that belong to a specific category (e.g., ITF Men)
    query4 = """
    SELECT c.competition_id, c.competition_name, c.type, c.gender
    FROM Competitions c
    JOIN Categories cat ON c.category_id = cat.category_id
    WHERE cat.category_name LIKE '%ITF Men%'
    """
    result4 = execute_query(query4, "Get competitions that belong to ITF Men category")
    
    # Query 5: Identify parent competitions and their sub-competitions
    query5 = """
    SELECT p.competition_id as parent_id, p.competition_name as parent_name,
           c.competition_id, c.competition_name
    FROM Competitions c
    JOIN Competitions p ON c.parent_id = p.competition_id
    """
    result5 = execute_query(query5, "Identify parent competitions and their sub-competitions")
    
    # Query 6: Analyze the distribution of competition types by category
    query6 = """
    SELECT cat.category_name, c.type, COUNT(c.competition_id) as count
    FROM Competitions c
    JOIN Categories cat ON c.category_id = cat.category_id
    GROUP BY cat.category_name, c.type
    ORDER BY cat.category_name, count DESC
    """
    result6 = execute_query(query6, "Analyze the distribution of competition types by category")
    
    # Query 7: List all competitions with no parent (top-level competitions)
    query7 = """
    SELECT competition_id, competition_name, type, gender
    FROM Competitions
    WHERE parent_id IS NULL
    """
    result7 = execute_query(query7, "List all competitions with no parent (top-level competitions)")
    
    return {
        "competitions_by_category": result1,
        "competition_count_by_category": result2,
        "doubles_competitions": result3,
        "itf_men_competitions": result4,
        "parent_child_competitions": result5,
        "competition_types_by_category": result6,
        "top_level_competitions": result7
    }

In [251]:
# Complexes SQL Queries
def run_complexes_queries():
    """Run and display results for complexes SQL queries"""
    # Query 1: List all venues along with their associated complex name
    query1 = """
    SELECT v.venue_id, v.venue_name, c.complex_name
    FROM Venues v
    JOIN Complexes c ON v.complex_id = c.complex_id
    """
    result1 = execute_query(query1, "List all venues along with their associated complex name")
    
    # Query 2: Count the number of venues in each complex
    query2 = """
    SELECT c.complex_name, COUNT(v.venue_id) as venue_count
    FROM Venues v
    JOIN Complexes c ON v.complex_id = c.complex_id
    GROUP BY c.complex_name
    ORDER BY venue_count DESC
    """
    result2 = execute_query(query2, "Count the number of venues in each complex")
    
    # Query 3: Get details of venues in a specific country (e.g., Chile)
    query3 = """
    SELECT venue_name, city_name, country_name
    FROM Venues
    WHERE country_name = 'Chile'
    """
    result3 = execute_query(query3, "Get details of venues in Chile")
    
    # Query 4: Identify all venues and their timezones
    query4 = """
    SELECT venue_name, timezone, country_name
    FROM Venues
    ORDER BY timezone
    """
    result4 = execute_query(query4, "Identify all venues and their timezones")
    
    # Query 5: Find complexes that have more than one venue
    query5 = """
    SELECT c.complex_name, COUNT(v.venue_id) as venue_count
    FROM Complexes c
    JOIN Venues v ON c.complex_id = v.complex_id
    GROUP BY c.complex_name
    HAVING COUNT(v.venue_id) > 1
    ORDER BY venue_count DESC
    """
    result5 = execute_query(query5, "Find complexes that have more than one venue")
    
    # Query 6: List venues grouped by country
    query6 = """
    SELECT country_name, COUNT(venue_id) as venue_count
    FROM Venues
    GROUP BY country_name
    ORDER BY venue_count DESC
    """
    result6 = execute_query(query6, "List venues grouped by country")
    
    # Query 7: Find all venues for a specific complex (e.g., Nacional)
    query7 = """
    SELECT v.venue_name, v.city_name, v.country_name
    FROM Venues v
    JOIN Complexes c ON v.complex_id = c.complex_id
    WHERE c.complex_name LIKE '%Nacional%'
    """
    result7 = execute_query(query7, "Find all venues for Nacional complex")
    
    return {
        "venues_by_complex": result1,
        "venue_count_by_complex": result2,
        "chile_venues": result3,
        "venues_by_timezone": result4,
        "complexes_multiple_venues": result5,
        "venues_by_country": result6,
        "nacional_complex_venues": result7
    }

In [252]:
def run_competitor_rankings_queries():
    """Run and display results for competitor rankings SQL queries"""
    # Query 1: Get all competitors with their rank and points
    query1 = """
    SELECT c.name, c.country, cr.rank, cr.points
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    ORDER BY cr.rank
    """
    result1 = execute_query(query1, "Get all competitors with their rank and points")
    
    # Query 2: Find competitors ranked in the top 5
    query2 = """
    SELECT c.name, c.country, cr.rank, cr.points
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    WHERE cr.rank <= 5
    ORDER BY cr.rank
    """
    result2 = execute_query(query2, "Find competitors ranked in the top 5")
    
    # Query 3: List competitors with no rank movement (stable rank)
    query3 = """
    SELECT c.name, c.country, cr.rank, cr.movement
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    WHERE cr.movement = 0
    ORDER BY cr.rank
    """
    result3 = execute_query(query3, "List competitors with no rank movement (stable rank)")
    
    # Query 4: Get the total points of competitors from a specific country (e.g., Croatia)
    query4 = """
    SELECT c.country, SUM(cr.points) as total_points, COUNT(c.competitor_id) as competitor_count
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    WHERE c.country = 'Croatia'
    GROUP BY c.country
    """
    result4 = execute_query(query4, "Get the total points of competitors from Croatia")
    
    # Query 5: Count the number of competitors per country
    query5 = """
    SELECT c.country, COUNT(c.competitor_id) as competitor_count
    FROM Competitors c
    GROUP BY c.country
    ORDER BY competitor_count DESC
    """
    result5 = execute_query(query5, "Count the number of competitors per country")
    
    # Query 6: Find competitors with the highest points in the current week
    query6 = """
    SELECT c.name, c.country, cr.rank, cr.points
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    ORDER BY cr.points DESC
    LIMIT 10
    """
    result6 = execute_query(query6, "Find competitors with the highest points in the current week")
    
    return {
        "competitors_with_rank": result1,
        "top_5_competitors": result2,
        "stable_rank_competitors": result3,
        "croatia_competitors_points": result4,
        "competitors_by_country": result5,
        "highest_points_competitors": result6
    }

In [253]:
# Function to create Streamlit app code
def create_streamlit_app():
    """Create Streamlit app code"""
    
    # Using triple single quotes for the outer string to avoid conflicts
    streamlit_code = '''
import streamlit as st
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Set page configuration
st.set_page_config(
    page_title="Tennis Analytics Dashboard",
    page_icon="🎾",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Connect to the database
@st.cache_resource
def get_connection():
    return sqlite3.connect('tennis_analytics.db')

# Function to execute SQL query and return DataFrame
@st.cache_data
def run_query(query):
    conn = get_connection()
    result = pd.read_sql_query(query, conn)
    return result

# Sidebar navigation
st.sidebar.title("Tennis Analytics")

page = st.sidebar.radio(
    "Navigate to",
    ["Dashboard", "Competitions", "Venues", "Competitors", "Search", "About"]
)

# Dashboard page
if page == "Dashboard":
    st.title("🎾 Tennis Analytics Dashboard")
    st.subheader("Unlocking Tennis Data with SportRadar API")
    
    # Create columns for summary statistics
    col1, col2, col3 = st.columns(3)
    
    # Total competitors
    total_competitors = run_query("SELECT COUNT(*) as count FROM Competitors").iloc[0]['count']
    col1.metric("Total Competitors", total_competitors)
    
    # Total countries
    total_countries = run_query("SELECT COUNT(DISTINCT country) as count FROM Competitors").iloc[0]['count']
    col2.metric("Countries Represented", total_countries)
    
    # Highest points
    highest_points = run_query("SELECT MAX(points) as max_points FROM Competitor_Rankings").iloc[0]['max_points']
    col3.metric("Highest Points", highest_points)
    
    st.markdown("---")
    
    # Create two columns for charts
    col1, col2 = st.columns(2)
    
    with col1:
        st.subheader("Competitors by Country")
        country_data = run_query("""
            SELECT country, COUNT(*) as count 
            FROM Competitors 
            GROUP BY country 
            ORDER BY count DESC 
            LIMIT 10
        """)
        fig = px.bar(country_data, x='country', y='count', color='count',
                    title="Top 10 Countries by Number of Competitors")
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        st.subheader("Competition Types Distribution")
        type_data = run_query("""
            SELECT type, COUNT(*) as count 
            FROM Competitions 
            GROUP BY type
        """)
        fig = px.pie(type_data, values='count', names='type', 
                    title="Distribution of Competition Types")
        st.plotly_chart(fig, use_container_width=True)
    
    # Rankings distribution
    st.subheader("Rankings Distribution")
    rank_data = run_query("""
        SELECT rank, points FROM Competitor_Rankings ORDER BY rank LIMIT 50
    """)
    fig = px.scatter(rank_data, x='rank', y='points', 
                    title="Relationship between Rank and Points (Top 50)",
                    labels={"rank": "Rank", "points": "Points"})
    st.plotly_chart(fig, use_container_width=True)

# Competitions page
elif page == "Competitions":
    st.title("Tennis Competitions")
    
    # Filters
    st.sidebar.header("Filters")
    
    # Get unique categories
    categories = run_query("SELECT DISTINCT category_name FROM Categories ORDER BY category_name")
    selected_category = st.sidebar.selectbox("Select Category", ["All"] + categories['category_name'].tolist())
    
    # Get unique types
    types = run_query("SELECT DISTINCT type FROM Competitions ORDER BY type")
    selected_type = st.sidebar.selectbox("Select Type", ["All"] + types['type'].tolist())
    
    # Get unique genders
    genders = run_query("SELECT DISTINCT gender FROM Competitions ORDER BY gender")
    selected_gender = st.sidebar.selectbox("Select Gender", ["All"] + genders['gender'].tolist())
    
    # Build query based on filters
    query = """
    SELECT c.competition_id, c.competition_name, cat.category_name, c.type, c.gender
    FROM Competitions c
    JOIN Categories cat ON c.category_id = cat.category_id
    WHERE 1=1
    """
    
    if selected_category != "All":
        query += f" AND cat.category_name = '{selected_category}'"
    
    if selected_type != "All":
        query += f" AND c.type = '{selected_type}'"
    
    if selected_gender != "All":
        query += f" AND c.gender = '{selected_gender}'"
    
    competitions = run_query(query)
    
    # Display competitions
    st.write(f"Showing {len(competitions)} competitions")
    st.dataframe(competitions)
    
    # Visualizations
    st.subheader("Competitions Analysis")
    
    col1, col2 = st.columns(2)
    
    with col1:
        # Competition count by category
        category_data = run_query("""
            SELECT cat.category_name, COUNT(c.competition_id) as count
            FROM Competitions c
            JOIN Categories cat ON c.category_id = cat.category_id
            GROUP BY cat.category_name
            ORDER BY count DESC
            LIMIT 10
        """)
        fig = px.bar(category_data, x='category_name', y='count', 
                    title="Top 10 Categories by Number of Competitions")
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        # Gender distribution
        gender_data = run_query("""
            SELECT gender, COUNT(*) as count 
            FROM Competitions 
            GROUP BY gender
        """)
        fig = px.pie(gender_data, values='count', names='gender', 
                    title="Gender Distribution in Competitions")
        st.plotly_chart(fig, use_container_width=True)

# Venues page
elif page == "Venues":
    st.title("Tennis Venues")
    
    # Filters
    st.sidebar.header("Filters")
    
    # Get unique countries
    countries = run_query("SELECT DISTINCT country_name FROM Venues ORDER BY country_name")
    selected_country = st.sidebar.selectbox("Select Country", ["All"] + countries['country_name'].tolist())
    
    # Build query based on filters
    query = """
    SELECT v.venue_id, v.venue_name, v.city_name, v.country_name, c.complex_name, v.timezone
    FROM Venues v
    JOIN Complexes c ON v.complex_id = c.complex_id
    WHERE 1=1
    """
    
    if selected_country != "All":
        query += f" AND v.country_name = '{selected_country}'"
    
    venues = run_query(query)
    
    # Display venues
    st.write(f"Showing {len(venues)} venues")
    st.dataframe(venues)
    
    # Visualizations
    st.subheader("Venues Analysis")
    
    col1, col2 = st.columns(2)
    
    with col1:
        # Venues by country
        country_data = run_query("""
            SELECT country_name, COUNT(*) as count 
            FROM Venues 
            GROUP BY country_name 
            ORDER BY count DESC 
            LIMIT 10
        """)
        fig = px.bar(country_data, x='country_name', y='count', color='count',
                    title="Top 10 Countries by Number of Venues")
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        # Venues by complex
        complex_data = run_query("""
            SELECT c.complex_name, COUNT(v.venue_id) as count
            FROM Venues v
            JOIN Complexes c ON v.complex_id = c.complex_id
            GROUP BY c.complex_name
            ORDER BY count DESC
            LIMIT 10
        """)
        fig = px.bar(complex_data, x='complex_name', y='count',
                    title="Top 10 Complexes by Number of Venues")
        st.plotly_chart(fig, use_container_width=True)
    
    # Map of venues
    st.subheader("Venues Map")
    st.write("Note: This is a placeholder for a map visualization. In a real implementation, you would need to geocode the venues.")

# Competitors page
elif page == "Competitors":
    st.title("Tennis Competitors")
    
    # Filters
    st.sidebar.header("Filters")
    
    # Get unique countries
    countries = run_query("SELECT DISTINCT country FROM Competitors ORDER BY country")
    selected_country = st.sidebar.selectbox("Select Country", ["All"] + countries['country'].tolist())
    
    # Rank range
    min_rank, max_rank = st.sidebar.slider("Rank Range", 1, 100, (1, 50))
    
    # Build query based on filters
    query = """
    SELECT c.name, c.country, c.country_code, cr.rank, cr.points, cr.movement, cr.competitions_played
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    WHERE cr.rank BETWEEN ? AND ?
    """
    
    params = [min_rank, max_rank]
    
    if selected_country != "All":
        query += " AND c.country = ?"
        params.append(selected_country)
    
    query += " ORDER BY cr.rank"
    
    # Execute query with parameters
    conn = get_connection()
    competitors = pd.read_sql_query(query, conn, params=params)
    
    # Display competitors
    st.write(f"Showing {len(competitors)} competitors")
    st.dataframe(competitors)
    
    # Visualizations
    st.subheader("Competitors Analysis")
    
    col1, col2 = st.columns(2)
    
    with col1:
        # Points vs Rank
        fig = px.scatter(competitors, x='rank', y='points', color='country',
                        hover_name='name', title="Points vs Rank")
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        # Movement analysis
        movement_data = competitors.copy()
        movement_data['movement_type'] = movement_data['movement'].apply(
            lambda x: 'Up' if x > 0 else ('Down' if x < 0 else 'No Change')
        )
        movement_counts = movement_data['movement_type'].value_counts().reset_index()
        movement_counts.columns = ['movement_type', 'count']
        
        fig = px.pie(movement_counts, values='count', names='movement_type',
                    title="Rank Movement Distribution")
        st.plotly_chart(fig, use_container_width=True)
    
    # Top 10 competitors
    st.subheader("Top 10 Competitors by Points")
    top_competitors = run_query("""
        SELECT c.name, c.country, cr.rank, cr.points
        FROM Competitors c
        JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
        ORDER BY cr.points DESC
        LIMIT 10
    """)
    
    fig = px.bar(top_competitors, x='name', y='points', color='country',
                text='rank', title="Top 10 Competitors by Points")
    fig.update_traces(texttemplate='Rank: %{text}', textposition='outside')
    st.plotly_chart(fig, use_container_width=True)

# Search page
elif page == "Search":
    st.title("Search Tennis Data")
    
    search_type = st.radio("Search Type", ["Competitor", "Competition", "Venue"])
    search_term = st.text_input("Enter search term")
    
    if search_term:
        if search_type == "Competitor":
            results = run_query(f"""
                SELECT c.name, c.country, cr.rank, cr.points
                FROM Competitors c
                JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
                WHERE c.name LIKE '%{search_term}%'
                ORDER BY cr.rank
            """)
            
            if not results.empty:
                st.write(f"Found {len(results)} competitors matching '{search_term}'")
                st.dataframe(results)
                
                # Show details for first match
                if len(results) > 0:
                    st.subheader(f"Details for {results.iloc[0]['name']}")
                    
                    competitor_name = results.iloc[0]['name']
                    competitor_details = run_query(f"""
                        SELECT c.name, c.country, c.country_code, c.abbreviation,
                               cr.rank, cr.points, cr.movement, cr.competitions_played
                        FROM Competitors c
                        JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
                        WHERE c.name = '{competitor_name}'
                    """)
                    
                    col1, col2 = st.columns(2)
                    
                    with col1:
                        st.write("**Basic Information**")
                        st.write(f"Name: {competitor_details.iloc[0]['name']}")
                        st.write(f"Country: {competitor_details.iloc[0]['country']} ({competitor_details.iloc[0]['country_code']})")
                        st.write(f"Abbreviation: {competitor_details.iloc[0]['abbreviation']}")
                    
                    with col2:
                        st.write("**Ranking Information**")
                        st.write(f"Current Rank: {competitor_details.iloc[0]['rank']}")
                        st.write(f"Points: {competitor_details.iloc[0]['points']}")
                        st.write(f"Movement: {competitor_details.iloc[0]['movement']}")
                        st.write(f"Competitions Played: {competitor_details.iloc[0]['competitions_played']}")
            else:
                st.warning(f"No competitors found matching '{search_term}'")
        
        elif search_type == "Competition":
            results = run_query(f"""
                SELECT c.competition_name, cat.category_name, c.type, c.gender
                FROM Competitions c
                JOIN Categories cat ON c.category_id = cat.category_id
                WHERE c.competition_name LIKE '%{search_term}%'
            """)
            
            if not results.empty:
                st.write(f"Found {len(results)} competitions matching '{search_term}'")
                st.dataframe(results)
            else:
                st.warning(f"No competitions found matching '{search_term}'")
        
        elif search_type == "Venue":
            results = run_query(f"""
                SELECT v.venue_name, v.city_name, v.country_name, c.complex_name
                FROM Venues v
                JOIN Complexes c ON v.complex_id = c.complex_id
                WHERE v.venue_name LIKE '%{search_term}%' OR v.city_name LIKE '%{search_term}%'
            """)
            
            if not results.empty:
                st.write(f"Found {len(results)} venues matching '{search_term}'")
                st.dataframe(results)
            else:
                st.warning(f"No venues found matching '{search_term}'")

# About page
elif page == "About":
    st.title("About Tennis Analytics")
    
    # Using markdown instead of multi-line string to avoid nested triple quotes
    st.markdown("""
    ## Game Analytics: Unlocking Tennis Data with SportRadar API
    
    This application provides comprehensive analytics for tennis data extracted from the SportRadar API.
    
    ### Features:
    
    - **Competition Analysis**: Explore tennis competitions, their categories, types, and gender distribution.
    - **Venue Exploration**: Discover tennis venues around the world and their associated complexes.
    - **Competitor Rankings**: Analyze player rankings, points, and performance metrics.
    - **Search Functionality**: Find specific competitors, competitions, or venues.
    """)
    
    st.markdown("""
    ### Data Sources:
    
    All data is sourced from the SportRadar API, which provides comprehensive sports data for various sports including tennis.
    """)
    
    st.markdown("""
    ### Technologies Used:
    
    - **Python**: Core programming language
    - **SQLite**: Database for storing and querying data
    - **Streamlit**: Web application framework
    - **Pandas**: Data manipulation and analysis
    - **Plotly**: Interactive data visualization
    """)
    
    st.markdown("""
    ### Project Information:
    
    This project was developed as part of a data analytics exercise to demonstrate skills in API integration, database management, and data visualization.
    """)

# Run the Streamlit app
if __name__ == "__main__":
    st.sidebar.info("Tennis Analytics Dashboard v1.0")
'''
    
    print("\nStreamlit App Code:")
    print(streamlit_code)
    
    # Save Streamlit app to a file
    with open('tennis_analytics_app.py', 'w') as f:
        f.write(streamlit_code)
    
    print("\nStreamlit app code saved to 'tennis_analytics_app.py'")

In [254]:
def main():
    """Main execution function"""
    print("Game Analytics: Unlocking Tennis Data with SportRadar API")
    print("=" * 60)
    
    # Step 1: Create database
    create_database()
    
    # Step 2: Collect data (commented out for demonstration)
    collect_competitions_data()
    collect_complexes_data()
    collect_doubles_rankings_data()
    
    # Step 3: Run SQL queries
    print("\nRunning SQL queries for analysis...")
    competitions_results = run_competitions_queries()
    complexes_results = run_complexes_queries()
    competitor_results = run_competitor_rankings_queries()
    
    # Step 4: Create Streamlit app
    create_streamlit_app()
    
    print("\nProject completed successfully!")

# Execute main function
if __name__ == "__main__":
    main()

Game Analytics: Unlocking Tennis Data with SportRadar API
Database created successfully!
Competitions data collected and stored successfully!
Complexes data collected and stored successfully!
Doubles rankings data collected and stored successfully!

Running SQL queries for analysis...

List all competitions along with their category name:


,competition_id,competition_name,category_name
0,sr:competition:620,Hopman Cup,Hopman Cup
1,sr:competition:660,World Team Cup,ATP
2,sr:competition:990,ATP Challenger Tour Finals,Challenger
3,sr:competition:1207,Championship International Series,WTA
4,sr:competition:2100,Davis Cup,Davis Cup
...,...,...,...
6017,sr:competition:46891,UTR Carvoeiro M03,UTR Men
6018,sr:competition:46901,UTR Newport Beach M07,UTR Men
6019,sr:competition:46903,UTR Boca Raton W07,UTR Women
6020,sr:competition:46905,UTR Melbourne W04,UTR Women



Count the number of competitions in each category:


,category_name,competition_count
0,ITF Men,2198
1,ITF Women,2032
2,Challenger,900
3,WTA,253
4,ATP,223
5,WTA 125K,172
6,UTR Men,83
7,UTR Women,75
8,Exhibition,32
9,Wheelchairs,16



Find all competitions of type 'doubles':


...


Count the number of competitors per country:


,country,competitor_count
0,USA,103
1,Japan,56
2,France,56
3,Great Britain,51
4,Australia,47
...,...,...
74,Burundi,1
75,Belarus,1
76,Armenia,1
77,Antigua and Barbuda,1



Find competitors with the highest points in the current week:


,name,country,rank,points
0,"Siniakova, Katerina",Czechia,1,10665
1,"Pavic, Mate",Croatia,1,9350
2,"Arevalo-Gonzalez, Marcelo",El Salvador,1,9350
3,"Townsend, Taylor",USA,2,8825
4,"Routliffe, Erin",New Zealand,3,8125
5,"Heliovaara, Harri",Finland,3,7590
6,"Patten, Henry",Great Britain,4,7590
7,"Ostapenko, Jelena",Latvia,4,6775
8,"Thompson, Jordan",Australia,5,6460
9,"Dabrowski, Gabriela",Canada,5,6385



Streamlit App Code:

import streamlit as st
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Set page configuration
st.set_page_config(
    page_title="Tennis Analytics Dashboard",
    page_icon="🎾",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Connect to the database
@st.cache_resource
def get_connection():
    return sqlite3.connect('tennis_analytics.db')

# Function to execute SQL query and return DataFrame
@st.cache_data
def run_query(query):
    conn = get_connection()
    result = pd.read_sql_query(query, conn)
    return result

# Sidebar navigation
st.sidebar.title("Tennis Analytics")

page = st.sidebar.radio(
    "Navigate to",
    ["Dashboard", "Competitions", "Venues", "Competitors", "Search", "About"]
)

# Dashboard page
if page == "Dashboard":
    st.title("🎾 Tennis Analytics Dashboard")
    st.subheader("Unlocking Tennis Data with SportRadar API")

   